### Практическое задание 8

## Вариант

### Классификаторы:

```
{
  "SVM linear",
  "Linear Discriminant Analysis",
  "Random Forest"
}
```


In [240]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_wine
from matplotlib import pyplot as plt
from sklearn.metrics import classification_report

from sklearn import svm
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.ensemble import RandomForestClassifier

from sklearn.preprocessing import MinMaxScaler

import numpy as np

In [241]:
# 1. Загрузить датасет WINE.
wine = load_wine(as_frame=True)
X = wine.data
Y = wine.target

mmsx = MinMaxScaler().fit_transform(X)

for idx, name in enumerate(wine.data):
    X[name] = mmsx[:, idx]

/tmp/ipykernel_13602/267608156.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[name] = mmsx[:, idx]


In [242]:
# 2. Подготовить выборки: обучающую 70% и тестовую 30%.
x_train, x_test, y_train, y_test = train_test_split(
    X,
    Y,
    test_size=0.3,
    stratify=Y)
x_train

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
18,0.831579,0.167984,0.598930,0.304124,0.413043,0.800000,0.757384,0.358491,0.457413,0.633106,0.609756,0.567766,1.000000
2,0.560526,0.320158,0.700535,0.412371,0.336957,0.627586,0.611814,0.320755,0.757098,0.375427,0.447154,0.695971,0.646933
151,0.463158,0.381423,0.598930,0.587629,0.456522,0.172414,0.215190,0.207547,0.268139,0.812287,0.000000,0.073260,0.144080
8,1.000000,0.177866,0.433155,0.175258,0.293478,0.627586,0.556962,0.301887,0.495268,0.334471,0.487805,0.578755,0.547076
127,0.200000,0.274704,0.759358,0.922680,0.239130,0.396552,0.400844,0.849057,0.425868,0.146758,0.398374,0.428571,0.134094
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26,0.621053,0.203557,0.673797,0.283505,0.250000,0.644828,0.548523,0.396226,0.328076,0.300341,0.357724,0.714286,0.654066
94,0.155263,0.247036,0.491979,0.381443,0.304348,0.703448,0.405063,0.075472,0.296530,0.168089,0.552846,0.619048,0.047789
156,0.739474,0.667984,0.545455,0.458763,0.206522,0.282759,0.103376,0.660377,0.362776,0.659556,0.073171,0.135531,0.144080
161,0.700000,0.498024,0.631016,0.484536,0.402174,0.293103,0.046414,0.698113,0.123028,0.392491,0.390244,0.201465,0.286733


In [243]:
# 3. Для указанных в задании классификаторов с использованием обучающей выборки выбрать 2 компоненты вектора признаков (2 признака), обеспечивающие наилучшее качество классификации.
from sklearn.feature_selection import (
    SelectKBest,
    f_classif
)


def get_best_features(x_train, y_train):
    x_train_selected = SelectKBest(
        f_classif, k=2).fit_transform(x_train, y_train)
    xt = x_train.copy().reset_index()
    feature_names = []
    for val in x_train_selected[0]:
        for name in x_train.columns:
            if xt[name][0] == val:
                feature_names.append(name)
                break
    return feature_names


cn = get_best_features(x_train, y_train)
cn

['flavanoids', 'proline']

In [244]:
# 4. Получить оценку качества классификации с использованием тестовой выборки.
x_train, x_test = x_train[cn], x_test[cn]
CLS_REPORT = []

In [245]:
# SVM linear
svm_clf = svm.SVC(kernel='linear', C=1, max_iter=20000)
svm_clf.fit(x_train, y_train)
print(classification_report(y_test, svm_clf.predict(x_test)))
CLS_REPORT.append(('SVM', svm_clf))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        18
           1       1.00      0.86      0.92        21
           2       0.83      1.00      0.91        15

    accuracy                           0.94        54
   macro avg       0.94      0.95      0.94        54
weighted avg       0.95      0.94      0.94        54



In [246]:
# Linear Discriminant Analysis
lda_clf = LDA(n_components=1)
_ = lda_clf.fit_transform(x_train, y_train)
print(classification_report(y_test, lda_clf.predict(x_test)))
CLS_REPORT.append(('LDA', lda_clf))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        18
           1       1.00      0.86      0.92        21
           2       0.83      1.00      0.91        15

    accuracy                           0.94        54
   macro avg       0.94      0.95      0.94        54
weighted avg       0.95      0.94      0.94        54



In [247]:
# Random Forest
rf_clf = RandomForestClassifier(n_estimators=30)
rf_clf.fit(x_train, y_train)
print(classification_report(y_test, rf_clf.predict(x_test)))
CLS_REPORT.append(('RandomForestClassifier', rf_clf))

              precision    recall  f1-score   support

           0       0.90      1.00      0.95        18
           1       0.94      0.76      0.84        21
           2       0.82      0.93      0.88        15

    accuracy                           0.89        54
   macro avg       0.89      0.90      0.89        54
weighted avg       0.89      0.89      0.89        54



In [252]:
# 5. Сравнить выбранные признаки для каждого из классификаторов и сделать вывод.
print(f'Выбранные признаки: {cn}')
for name, clf in CLS_REPORT:
    print(f'---- {name} ----')
    print(classification_report(y_test, clf.predict(x_test)))
    print(f'---- ------ ----')
print()
print('---- Вывод ----')
print(
    f"""
Наилучшую точность продемонстрировали SVM и LDA классификаторы
""")

Выбранные признаки: ['flavanoids', 'proline']
---- SVM ----
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        18
           1       1.00      0.86      0.92        21
           2       0.83      1.00      0.91        15

    accuracy                           0.94        54
   macro avg       0.94      0.95      0.94        54
weighted avg       0.95      0.94      0.94        54

---- ------ ----
---- LDA ----
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        18
           1       1.00      0.86      0.92        21
           2       0.83      1.00      0.91        15

    accuracy                           0.94        54
   macro avg       0.94      0.95      0.94        54
weighted avg       0.95      0.94      0.94        54

---- ------ ----
---- RandomForestClassifier ----
              precision    recall  f1-score   support

           0       0.90      1.00      0.9